# Graph Traversal

## DFS

In [ ]:
graph = {
    0: [1, 3],
    1: [2, 4, 6],
    2: [1, 9, 0],
    3: [4, 0, 8],
    4: [3, 7],
    5: [9, 2],
    8: [2, 9]
}


def dfs_traverse(graph, start=0):
    """
    Performs DFS traversal on a directed graph starting from the given node.

    Args:
        graph (dict[int, list[int]]): Adjacency list representing the graph.
        start (int): Starting node for DFS traversal.

    Returns:
        list[int]: List of nodes in the order they were visited.
    """
    visited = set()
    result = []

    def dfs(node):
        visited.add(node)
        result.append(node)
        for neighbor in graph.get(node, []):
            if neighbor not in visited:
                dfs(neighbor)

    dfs(start)
    return result


print(dfs_traverse(graph))

## BFS

In [ ]:


graph = {
    0: [1, 3],
    1: [2, 4, 6],
    2: [1, 9, 0],
    3: [4, 0, 8],
    4: [3, 7],
    5: [9, 2],
    8: [2, 9]
}


def bfs_traverse(graph, start=0):
    """
    Performs BFS traversal on a directed graph starting from the given node.

    Args:
        graph (dict[int, list[int]]): Adjacency list representing the graph.
        start (int): Starting node for BFS traversal.

    Returns:
        list[int]: List of nodes in the order they were visited.
    """
    bfs_queue = deque([start])
    result = []
    visited = {start}

    while bfs_queue:
        vertex = bfs_queue.popleft()
        result.append(vertex)

        for child in graph.get(vertex, []):
            if child not in visited:
                visited.add(child)
                bfs_queue.append(child)

    return result


print(bfs_traverse(graph))

# Path Finding

## Find all paths from start

In [ ]:
graph = {
    'A': ['B', 'C'],
    'B': ['C', 'D'],
    'C': ['D'],
    'D': ['E'],
    'E': []
}


def find_all_paths_from_start(graph, start):
    visiting = set()
    result = []

    def backtrack(path: List[str], node: str):
        if node in visiting:
            return

        path.append(node)
        visiting.add(node)

        if not graph.get(node, []):
            result.append(path.copy())
            path.pop()
            visiting.remove(node)
            return

        for child in graph.get(node, []):
            backtrack(path, child)

        path.pop()
        visiting.remove(node)

    backtrack([], start)
    return result


print(find_all_paths_from_start(graph, 'A'))

## Find all paths to target

In [ ]:


graph = {
    'A': ['B', 'C'],
    'B': ['C', 'D'],
    'C': ['D'],
    'D': ['E'],
    'E': []
}


def find_all_paths_to_target(graph, start, target):
    """
    Finds all acyclic paths from the start node to the target node in a directed graph.

    Args:
        graph (dict[str, list[str]]): Adjacency list representing the directed graph.
        start (str): The starting node.
        target (str): The destination node.

    Returns:
        list[list[str]]: A list of all valid paths from start to target.
    """
    result = []
    visiting = set()

    def backtrack(path, node):
        if node in visiting:
            return

        path.append(node)
        visiting.add(node)

        if node == target:
            result.append(path.copy())
            path.pop()
            visiting.remove(node)
            return

        for child in graph.get(node, []):
            backtrack(path, child)
        path.pop()
        visiting.remove(node)

    backtrack([], start)
    return result


print(find_all_paths_to_target(graph, 'A', 'E'))

## Find the shortest path

In [ ]:



def find_shortest_path_bfs(graph, start, target):
    queue = deque([start])
    parent = {start: None}
    while queue:
        node = queue.popleft()
        if node == target:
            break
        for neighbor in graph.get(node, []):
            if neighbor not in parent:
                parent[neighbor] = node
                queue.append(neighbor)
    if target not in parent:
        return None
    # 回溯路径
    path = []
    while target is not None:
        path.append(target)
        target = parent[target]
    return path[::-1]

# Reachability

# Connectivity

# Topological Sort

## Topological Sort using DFS (Post-order Insertion)

In [ ]:
from typing import List


# Topological Sort using DFS (post-order insertion)
# Applicable for DAGs (Directed Acyclic Graphs)
# Produces a valid topological ordering
def topological_sort_dfs(n: int, edges: List[List[int]]) -> List[int]:
    graph = defaultdict(list)
    for u, v in edges:
        graph[u].append(v)

    visited = set()
    stack = []

    def dfs(node):
        if node in visited:
            return

        visited.add(node)

        for child in graph[node]:
            dfs(child)
        stack.append(node)

    for node in range(n):
        dfs(node)

    # Reverse post-order to get topological order
    return stack[::-1]


## BFS 拓扑排序（Kahn’s Algorithm）

In [ ]:
from collections import deque


def topological_sort(n, edges):
    graph = defaultdict(list)
    indegree = [0] * n

    for u, v in edges:
        graph[u].append(v)
        indegree[v] += 1

    queue = deque([i for i in range(n) if indegree[i] == 0])
    topo_order = []

    while queue:
        node = queue.popleft()
        topo_order.append(node)
        for neighbor in graph[node]:
            indegree[neighbor] -= 1
            if indegree[neighbor] == 0:
                queue.append(neighbor)

    if len(topo_order) != n:
        raise ValueError("Graph has a cycle")

    return topo_order

# Shortest Path

# Cycle Detection

## Cycle Detection using 3-Color Marking (DFS)

使用三色标记法（0=未访问，1=访问中，2=已完成）检测有向图中是否存在环。

In [ ]:
from collections import defaultdict

from enum import IntEnum
from typing import List, Tuple


class State(IntEnum):
    UNVISITED = 0
    VISITING = 1
    VISITED = 2


def build_graph(edges: List[Tuple[int, int]]) -> dict:
    graph = defaultdict(list)
    for u, v in edges:
        graph[u].append(v)
    return graph


def has_cycle_3color(n: int, edges: List[Tuple[int, int]]) -> bool:
    graph = build_graph(edges)
    state = defaultdict(lambda: State.UNVISITED)

    def backtrack(node: int) -> bool:
        if state[node] == State.VISITING:
            return True
        # In fact, it doesn't matter whether we check State.VISITING or State.VISITED first — the result is the same. 但是我总感觉不太对，因为我以为State.VISITED是只要访问过，就叫State.VISITED。我以为也许经过这个节点，也许会有其他的路径会产生环。所以如果仅仅是因为这个节点被访问过就返回，可能会漏掉环。但是实际上并不是这样，实际上State.VISITED说明一个节点的所有子节点的路径都被访问过了，没有cycle，所以一旦发现是State.VISITED就不访问一个节点是比State.VISITING的节点更安全的，只有State.VISITING的节点才会出现cycle。
        if state[node] == State.VISITED:
            return False

        state[node] = State.VISITING
        for child in graph[node]:
            if backtrack(child):
                return True
        state[node] = State.VISITED
        return False

    for node in range(n):
        if state[node] == State.UNVISITED:
            if backtrack(node):
                return True

    return False